In [1]:
from bs4 import BeautifulSoup
import urllib.request
import time
import json

In [2]:
def get_crime_zipcode_mapping():
    list_urls = get_all_urls()
    ret_dict = {}
    for url in list_urls:
        wp = urllib.request.urlopen(url)
        soup = BeautifulSoup(wp, 'html.parser')
        tables = soup.findAll("table")
        tags = tables[1].findAll("td")
        property_crime = tags[-2].string
        violent_crime = tags[-5].string
        url_split = url.split("/")
        zipcode = url_split[-1]
        city_name = url_split[-2]
        if (property_crime == None):
            final_score = 105.6
        else:
            final_score = 1.5*float(violent_crime) + float(property_crime)
        if (city_name in ret_dict):
            (ret_dict[city_name])[zipcode] = final_score
        else:
            ret_dict[city_name] = {zipcode : final_score}
    return ret_dict

In [3]:
def get_zipcodes_by_city(city_name):
    with open('./zipcodes_by_city.json') as data_file:    
        data = json.load(data_file)
    return data[city_name]

In [4]:
base_url = "http://www.bestplaces.net/crime/zip-code/illinois"
city_list = ['Boston','San Francisco', 'Chicago', 'New York']
city_list_to_url = {"San Francisco": 'san_francisco', "Boston": "boston", "Chicago": "chicago", "New York": "new_york"}

In [5]:
def get_all_urls():
    ret_urls = []
    for city in city_list:
        city_zipcodes = get_zipcodes_by_city(city)
        for zipcode in city_zipcodes:
            parameters = "/{city_code}/{zip_code}".format(city_code=city_list_to_url[city], zip_code=zipcode)
            url = base_url + parameters
            ret_urls.append(url)
    return ret_urls

In [6]:
data = get_crime_zipcode_mapping()

In [7]:
def get_max_value_dict():
    max_value = -1
    for key in data.keys():
        new_dict = data[key]
        curr_max_value = new_dict[max(new_dict, key=new_dict.get)] 
        if (max_value < curr_max_value):
            max_value = curr_max_value
            new_dict = data[key]
    return max_value

In [8]:
max_value_in_dict = get_max_value_dict()
for key in data.keys():
    curr_dict = data[key]
    for key,value in curr_dict.items():
        value = value/max_value_in_dict
        curr_dict[key] = value

In [9]:
with open('./zipcode_crime.json', 'w') as outfile:
    json.dump(data, outfile)